In [5]:
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [10]:
"""
json schema: create description of the function
eventhough the function does not exist, llm still generates appropriate arguments but no content

compare with using llm twince in Quiz GPT
"""

function = {
    "name": "create_quiz", # name of the function
    "description": "function that takes a list of questions and answers and returns a quiz", 
    "parameters": { # arguments of the function
        "type": "object",
        "properties": { 
            "questions": { 
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": { 
                        "question": { 
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={ # force the model to use function ("auto" if you want to let the model decide)
        "name": "create_quiz", # name of the function
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")
chain = prompt | llm
answer = chain.invoke({"city": "rome"})

args = answer.additional_kwargs["function_call"]["arguments"]
args = json.loads(args) # since we definced parameters as json
args

{'questions': [{'question': 'What year was Rome founded?',
   'answers': [{'answer': '753 BC', 'correct': True},
    {'answer': '476 AD', 'correct': False},
    {'answer': '1000 BC', 'correct': False}]},
  {'question': 'Who was the first emperor of Rome?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False}]},
  {'question': 'What famous structure in Rome was built by the ancient Romans?',
   'answers': [{'answer': 'Eiffel Tower', 'correct': False},
    {'answer': 'Colosseum', 'correct': True},
    {'answer': 'Big Ben', 'correct': False}]}]}